# Sheep Illumina chip aligned
I've aligned with megablast the sheep chip probeset to the oldest `OAR4` assembly. I want to compare my results with result I get from dbSNP

In [1]:
from collections import defaultdict

from src.features.smarterdb import global_connection, VariantSheep
from src.data.common import AssemblyConf

import pandas as pd

In [2]:
conn = global_connection()
dbSNP152 = AssemblyConf(version="Oar_v4.0", imported_from="dbSNP152")

First, get my data aligned and set `snp_name` as a index:

In [3]:
results = pd.read_csv("ovinesnp50-genome-assembly-oar-v4-0.csv-GCA_000298735.2_Oar_v4.0_genomic.fna.blastn.csv")
results.set_index("snp_name", inplace=True)
results.head()

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt
snp_name,,,,,,,,,
250506CS3900065000002_1238.1,15,5859890,C/T,T/C,T/C,BOT,forward,C,T
250506CS3900140500001_312.1,23,26243215,C/T,T/C,T/C,BOT,forward,C,T
250506CS3900176800001_906.1,7,81590897,C/T,T/C,T/C,BOT,forward,C,T
250506CS3900211600001_1041.1,16,41363310,G/T,T/G,T/G,BOT,forward,G,T
250506CS3900218700001_1294.1,2,148834939,C/T,T/C,T/C,BOT,forward,C,T


Next load errors: If I can't place a SNP on a chromosome, I will have no position in results table and I will have a reason in this table:

In [4]:
errors = pd.read_csv("ovinesnp50-genome-assembly-oar-v4-0.csv-GCA_000298735.2_Oar_v4.0_genomic.fna.blastn.err")
errors.set_index("snp_name", inplace=True)
errors.head()

,illumina,illumina_strand,reason
snp_name,,,
250506CS3900371000001_1255.1,T/C,BOT,Cannot determine a unique SNP position
CR_594.1,T/C,BOT,Too many alignments after filtering
CR_816.1,T/C,BOT,Too many alignments after filtering
CytB_1406.1,T/C,BOT,No valid alignments after filtering
CytB_1505.1,A/G,TOP,No valid alignments after filtering


Ok, now get my Sheep variants and focus on *NCBI* data: I could have more variants than *NCBI* if there are probes more recent than dbSNP152:

In [5]:
ncbi_variants = VariantSheep.objects.filter(chip_name="IlluminaOvineSNP50", locations__match=dbSNP152._asdict(), rs_id__exists=True)
ncbi_variants.count()

48224

Ok now extract dbSNP locations from my `ncbi_variants`:

In [6]:
tmp = defaultdict(list)

for variant in ncbi_variants:
    location = variant.get_location(**dbSNP152._asdict())
    tmp["snp_name"].append(variant.name)
    tmp["rs_id"].append(",".join(variant.rs_id))
    tmp["ncbi_chrom"].append(location.chrom)
    tmp["ncbi_position"].append(location.position)
    
    
ncbi_locations = pd.DataFrame.from_dict(tmp)
ncbi_locations.set_index('snp_name', inplace=True)
ncbi_locations.head()

,rs_id,ncbi_chrom,ncbi_position
snp_name,,,
OAR16_74703566.1,"rs406297509,rs1087899539",16,68777502
OAR5_38485386.1,"rs402039066,rs1093088087",5,34727924
OAR5_87409839_X.1,rs119102657,5,79472633
250506CS3900414400001_1178.1,rs119102699,1,103285485
s05189.1,rs159412897,1,121010442


Ok, merge these data in a new dataframe. Get rid of SNPs not in *NCBI*, mind that since `ncbi_position` could have *NA* values, need to be converted as *integer*:

In [7]:
tmp = results.merge(ncbi_locations, how="left", on="snp_name")
ncbi_results = tmp[tmp['ncbi_chrom'].isna() == False].astype({'ncbi_position':'int'})
ncbi_results.head()

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position
snp_name,,,,,,,,,,,,
250506CS3900065000002_1238.1,15,5859890,C/T,T/C,T/C,BOT,forward,C,T,rs55630613,15,5859890
250506CS3900140500001_312.1,23,26243215,C/T,T/C,T/C,BOT,forward,C,T,rs55630642,23,26243215
250506CS3900176800001_906.1,7,81590897,C/T,T/C,T/C,BOT,forward,C,T,rs55630654,7,81590897
250506CS3900211600001_1041.1,16,41363310,G/T,T/G,T/G,BOT,forward,G,T,rs55630658,16,41363310
250506CS3900218700001_1294.1,2,148834939,C/T,T/C,T/C,BOT,forward,C,T,rs55630663,2,148834939


Ok focus on the differences between my alignment and NCBI:

In [8]:
differences = ncbi_results.query("chrom != ncbi_chrom | position != ncbi_position")
differences.head()

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position
snp_name,,,,,,,,,,,,
250506CS3900371000001_1255.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs417377113,11,35291132
DU172264_319.1,25,20596182,A/G,T/C,A/G,BOT,reverse,G,A,rs55632153,25,20596183
DU175804_598.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs409850824,13,12526490
DU179070_177.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs55628106,1,111700859
DU186191_327.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs55631284,26,4325569


In [9]:
differences.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2619 entries, 250506CS3900371000001_1255.1 to s75909.1
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   chrom             2619 non-null   object
 1   position          2619 non-null   int64 
 2   alleles           621 non-null    object
 3   illumina          2619 non-null   object
 4   illumina_forward  621 non-null    object
 5   illumina_strand   2619 non-null   object
 6   strand            621 non-null    object
 7   ref               621 non-null    object
 8   alt               621 non-null    object
 9   rs_id             2619 non-null   object
 10  ncbi_chrom        2619 non-null   object
 11  ncbi_position     2619 non-null   int64 
dtypes: int64(2), object(10)
memory usage: 266.0+ KB


I have ~2600 differences between NCBI, let's focus on different chromosome types:

In [10]:
differences["chrom"].value_counts()

0                 1998
1                  104
3                  103
2                  102
X                   29
5                   20
6                   19
18                  19
13                  19
9                   18
4                   18
19                  17
10                  11
11                  11
22                  11
15                  11
26                  11
14                  11
7                   10
12                  10
24                   9
16                   9
8                    9
17                   9
25                   8
20                   8
23                   7
21                   7
AMGL02044162.1       1
Name: chrom, dtype: int64

Ok, tell me how many SNPs I can't place, while *NCBI* can:

In [11]:
not_placed = differences[differences["chrom"] == '0']
print(f"There are {not_placed.shape[0]} SNPs that I can't map to genome")
not_placed.merge(errors["reason"], how="left", on="snp_name").sort_values("reason")

There are 1998 SNPs that I can't map to genome


,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position,reason
snp_name,,,,,,,,,,,,,
s10981.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs417802007,1,214819729,Allele doesn't match to reference
OAR10_2262648.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs418784080,10,4415179,Allele doesn't match to reference
s36152.1,0,0,NaN,T/G,NaN,BOT,NaN,NaN,NaN,rs398134079,13,34476763,Allele doesn't match to reference
OAR17_17335672.1,0,0,NaN,A/C,NaN,TOP,NaN,NaN,NaN,rs412691732,17,15594992,Allele doesn't match to reference
s08771.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs428537703,3,21069340,Allele doesn't match to reference
...,...,...,...,...,...,...,...,...,...,...,...,...,...
OAR2_251630246.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs412796743,2,238241826,Too many alignments after filtering
s19848.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs416766244,5,5112380,Too many alignments after filtering
s35302.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs399143327,21,25155055,Too many alignments after filtering


Well, there are a lot of SNPs I cannot match. Group them by reason:

In [12]:
not_placed.merge(errors["reason"], how="left", on="snp_name")["reason"].value_counts()

No valid alignments after filtering       1839
Allele doesn't match to reference           76
Too many alignments after filtering         57
Cannot determine a unique SNP position      26
Name: reason, dtype: int64

In [13]:
tmp = not_placed.merge(errors["reason"], how="left", on="snp_name")
tmp[tmp["reason"] == "No valid alignments after filtering"]

,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position,reason
snp_name,,,,,,,,,,,,,
DU179070_177.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs55628106,1,111700859,No valid alignments after filtering
DU191809_420.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,"rs428367938,rs405711682",1,186920065,No valid alignments after filtering
DU192841_628.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs55632389,23,19282729,No valid alignments after filtering
DU206491_155.1,0,0,NaN,A/C,NaN,TOP,NaN,NaN,NaN,rs429215508,14,1506257,No valid alignments after filtering
DU211594_392.1,0,0,NaN,A/C,NaN,TOP,NaN,NaN,NaN,rs55630233,X,126495510,No valid alignments after filtering
...,...,...,...,...,...,...,...,...,...,...,...,...,...
s75319.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs399172369,7,5799889,No valid alignments after filtering
s75496.1,0,0,NaN,A/G,NaN,TOP,NaN,NaN,NaN,rs415918180,1,260798427,No valid alignments after filtering
s75759.1,0,0,NaN,T/C,NaN,BOT,NaN,NaN,NaN,rs429941367,6,108740789,No valid alignments after filtering


Are then any *SNP* which I map to a different position than ncbi??

In [14]:
different = differences.query("chrom != '0' and ncbi_chrom != '0'")
print(f"There are {different.shape[0]} SNPs that I can map to a different position")
different.merge(errors["reason"], how="left", on="snp_name").sort_values("reason")

There are 615 SNPs that I can map to a different position


,chrom,position,alleles,illumina,illumina_forward,illumina_strand,strand,ref,alt,rs_id,ncbi_chrom,ncbi_position,reason
snp_name,,,,,,,,,,,,,
DU172264_319.1,25,20596182,A/G,T/C,A/G,BOT,reverse,G,A,rs55632153,25,20596183,NaN
DU206996_498.1,5,33118535,A/C,T/G,A/C,BOT,reverse,A,C,"rs403872294,rs421000549",5,33118534,NaN
DU240765_244.1,X,121221683,C/T,A/G,T/C,TOP,reverse,C,T,rs55630244,X,121221682,NaN
DU266660_200.1,2,119804090,C/T,A/G,T/C,TOP,reverse,C,T,rs55630890,2,119804089,NaN
DU286106_170.1,26,3412784,C/T,A/G,T/C,TOP,reverse,T,C,rs55628429,26,3412783,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
s75063.1,7,97723185,C/T,A/G,T/C,TOP,reverse,C,T,rs402289455,7,97723186,NaN
s75140.1,1,190436466,A/G,T/C,A/G,BOT,reverse,A,G,rs411885717,1,190436464,NaN
s75196.1,12,50493324,C/T,A/G,T/C,TOP,reverse,C,T,rs416182289,12,50493323,NaN
